### Import packages that we need

In [1]:
import hyperspy.api as hs

In [2]:
import exspy

In [3]:
import matplotlib.pyplot as plt
%matplotlib qt
import numpy as np
import IPython.utils.io

In [4]:
import espm

In [5]:
import json

In [6]:
from espm.conf import NUMBER_PERIODIC_TABLE

### Define useful functions

In [7]:
def symbol_list () : 
    symbol_list = []
    with open(NUMBER_PERIODIC_TABLE,"r") as f : 
            NPT = json.load(f)["table"]
    for num in NPT.keys() : 
        symbol_list.append(NPT[num]["symbol"])
    return symbol_list

In [8]:
def num_to_symbol(num):
    r"""
    Converts number to atomic symbol.

    Parameters
    ----------
    num : str
        Number to be converted to atomic symbol. E. g. "1" return "H"
    
    Returns
    -------
    element : str
        Corresponding atomic symbol.
    """
    d = {str(i+1):el for i,el in enumerate(symbol_list())} 
    try:
        return d[num.split("_")[0]]+"_"+num.split("_")[1]
    except:
        try:
            return d[num]
        except:
            return num

In [35]:
a = np.array([[1.]])
a.shape
a.sum()

1.0

In [9]:
def get_full_el_list(dataset):
    els = dataset.metadata.EDS_model.elements.copy()
    els_names = [num_to_symbol(el) for el in els]
    return els_names

In [10]:
def plot_data_model_ROI(dataset):
    r"""
    Plots ESPM EDXS model fit results on the experimental data, summed over the chosen region of interest.

    Parameters
    ----------
    None : 
        The data are taken from a previous decomposition.
        
    Returns
    -------
    None
    """
        
    W = dataset.learning_results.decomposition_algorithm.W_
    G = dataset.learning_results.decomposition_algorithm.G_
    H = dataset.learning_results.decomposition_algorithm.H_

    WH = np.matmul(W, H)

    contributions = [hs.signals.Signal1D((G[:, [i]] @ (WH)[[i], :]).T.reshape(dataset.data.shape)) for i in range(G.shape[1])]
    contributions.append(hs.signals.Signal1D((np.matmul(G, WH)).T.reshape(dataset.data.shape)))

    titles = get_full_el_list(dataset) + ["Background 1", "Background 2", "Full Model"]
    for i, c in enumerate(contributions):
        for a, b in zip(c.axes_manager._axes, dataset.axes_manager._axes):
            a.update_from(b)
        c.metadata.General.title = titles[i]

    fig, ax = plt.subplots()
    dataset.plot()

    roi = hs.roi.RectangularROI(left = 0, top = 0, right = dataset.axes_manager[1].size, bottom = dataset.axes_manager[0].size)
        
    imr = roi.interactive(dataset, color = 'green').sum(axis = 0).sum(axis = 0)
    contributions_roi = [roi.interactive(g, None).sum(axis = 0).sum(axis = 0) for g in contributions]

    spectra = [imr] + contributions_roi
    lines = []
        
    line, = ax.plot(imr.data, label = imr.metadata.General.title, linestyle = "-")
    lines.append(line)
        
    for spectrum in contributions_roi:
        line, = ax.plot(spectrum.data, label = spectrum.metadata.General.title, linestyle = "--")
        lines.append(line)
        
    ax.legend()

    def update_plot(*args, **kwargs):
        imr = roi.interactive(dataset, color = 'green').sum(axis = 0).sum(axis = 0)
        contributions_roi = [roi.interactive(g, None).sum(axis = 0).sum(axis = 0) for g in contributions]

        all_data = [imr] + contributions_roi
        for line, new_data in zip(lines, all_data):
            line.set_ydata(new_data.data)

        ax.relim()
        ax.autoscale_view()
        fig.canvas.draw_idle()

    roi.events.changed.connect(update_plot)
    update_plot()
    plt.show()

    return

#### Load data from Velox

In [11]:
# Choose file
input_filename = 'spectrum_2.hspy'

In [12]:
# Load Velox dataset with separate detector quadrant spectrum images 
spectrum_2 = hs.load(input_filename)

#### Look at axes_manager of haadf and spim, correct any differences

### Set up to quantify spectrum using espm

In [13]:
spectrum_2.change_dtype("float32")

In [14]:
spectrum_2.set_signal_type("EDSespm")

c:\Users\adteurtrie\AppData\Local\anaconda3\envs\espm\Lib\site-packages\exspy\misc\material.py:29: VisibleDeprecationWarning: This module is deprecated, use `exspy.material` instead. It will be removed in exspy 1.0.
  warnings.warn(
c:\Users\adteurtrie\AppData\Local\anaconda3\envs\espm\Lib\site-packages\exspy\misc\eds\utils.py:15: VisibleDeprecationWarning: This module is deprecated, use `exspy.utils.eds` instead. It will be removed in exspy 1.0.
  warnings.warn(


In [15]:
# Set analysis parameters for spectrum_2, including element list, detector type and X-ray database
spectrum_2.set_analysis_parameters(beam_energy=200,elements = ['C','O', 'Sc', 'Cu', 'Dy'],
                                  detector_type='SDD_efficiency.txt',
                                   thickness = 2e-5,
                                   elevation_angle=18.0,
                                   azimuth_angle=45.0,
                                   density = 6.9,
                                   tilt_stage=0.0,
                                  xray_db='200keV_xrays.json')

Warning! Optional argument: ['width_slope'] specified by user but not used
Warning! Optional argument: ['width_intercept'] specified by user but not used
Warning! Optional argument: ['take_off_angle'] specified by user but not used
Warning! Optional argument: ['EDS']['width_slope'] specified by user but not used
Warning! Optional argument: ['EDS']['width_intercept'] specified by user but not used
Warning! Optional argument: ['EDS']['take_off_angle'] specified by user but not used
Warning! Optional argument: ['Detector']['EDS']['width_slope'] specified by user but not used
Warning! Optional argument: ['Detector']['EDS']['width_intercept'] specified by user but not used
Warning! Optional argument: ['Detector']['EDS']['take_off_angle'] specified by user but not used


In [16]:
from exspy._misc.eds.utils import get_xray_lines_near_energy

In [17]:
spectrum_2.metadata.Sample.density = 6.9
# spectrum_2.metadata.Sample.thickness = 5.9e-5

In [18]:
spectrum_2.metadata.Acquisition_instrument.TEM.Detector.EDS.take_off_angle = 7

In [19]:
spectrum_2.metadata

├── Acquisition_instrument
│   └── TEM
│       ├── Detector
│       │   └── EDS
│       │       ├── azimuth_angle = 45.0
│       │       ├── elevation_angle = 18.0
│       │       ├── energy_resolution_MnKa = 130.0
│       │       ├── number_of_frames = 250
│       │       ├── take_off_angle = 7
│       │       ├── type = SDD_efficiency.txt
│       │       ├── width_intercept = 0.065
│       │       └── width_slope = 0.01
│       ├── Stage
│       │   ├── tilt_alpha = 0.0
│       │   ├── tilt_beta = -0.44
│       │   ├── x = -0.000134
│       │   ├── y = 1e-05
│       │   └── z = -0.000163
│       ├── beam_energy = 200
│       ├── camera_length = 115.9
│       ├── magnification = 7200000.0
│       └── microscope = Titan
├── General
│   ├── FileIO
│   │   ├── 0
│   │   │   ├── hyperspy_version = 2.1.1
│   │   │   ├── io_plugin = rsciio.emd
│   │   │   ├── operation = load
│   │   │   └── timestamp = 2024-11-18T16:38:16.357404+01:00
│   │   ├── 1
│   │   │   ├── hyperspy_version = 2.1.1
│   │   │   ├── io_plugin = rsciio.hspy
│   │   │   ├── operation = save
│   │   │   └── timestamp = 2024-11-19T14:23:59.740064+01:00
│   │   └── 2
│   │       ├── hyperspy_version = 2.2.0
│   │       ├── io_plugin = rsciio.hspy
│   │       ├── operation = load
│   │       └── timestamp = 2024-12-06T16:15:50.073937+01:00
│   ├── date = 2024-10-11
│   ├── original_filename = DSO.emd
│   ├── time = 11:00:13
│   ├── time_zone = CET
│   └── title = EDS
├── Sample
│   ├── density = 6.9
│   ├── elements = ['C', 'O', 'Sc', 'Cu', 'Dy']
│   └── thickness = 2e-05
├── Signal
│   └── signal_type = EDSespm
└── xray_db = 200keV_xrays.json

In [20]:
elements_dict = {'Cu' : 3.0}

In [21]:
# Build the G  matrix that is needed for espm decomposition
spectrum_2.build_G()

In [22]:
fw = spectrum_2.set_fixed_W({'p' : {'O' : 0.6}})

In [22]:
# Run espm in such a way as to quantify spectrum_2 as a single phase
from espm.estimators import SmoothNMF
#est = espm.estimators.SmoothNMF(n_components = 1, tol = 1e-6, max_iter = 500, G = spectrum_2.model, hspy_comp = True)
est = espm.estimators.SmoothNMF(n_components = 1, tol = 1e-6, max_iter = 5000, G = spectrum_2.model, hspy_comp = True,simplex_W=True, dicotomy_tol=1e-12)
spectrum_2.decomposition(algorithm = est)

It 10 / 5000: loss 1.317077e+02,  158.736 it/s
It 20 / 5000: loss 1.312959e+02,  163.942 it/s
It 30 / 5000: loss 1.312238e+02,  193.554 it/s
It 40 / 5000: loss 1.311705e+02,  220.996 it/s
It 50 / 5000: loss 1.311132e+02,  248.731 it/s
It 60 / 5000: loss 1.310490e+02,  270.224 it/s
It 70 / 5000: loss 1.309766e+02,  281.853 it/s
It 80 / 5000: loss 1.308941e+02,  294.815 it/s
It 90 / 5000: loss 1.307993e+02,  305.752 it/s
It 100 / 5000: loss 1.306895e+02,  311.179 it/s
It 110 / 5000: loss 1.305608e+02,  321.301 it/s
It 120 / 5000: loss 1.304079e+02,  331.167 it/s
It 130 / 5000: loss 1.302236e+02,  337.330 it/s
It 140 / 5000: loss 1.299972e+02,  345.354 it/s
It 150 / 5000: loss 1.297132e+02,  351.804 it/s
It 160 / 5000: loss 1.293474e+02,  354.471 it/s
It 170 / 5000: loss 1.288602e+02,  360.648 it/s
It 180 / 5000: loss 1.281836e+02,  366.318 it/s
It 190 / 5000: loss 1.271908e+02,  367.951 it/s
It 200 / 5000: loss 1.256221e+02,  372.874 it/s
It 210 / 5000: loss 1.228819e+02,  377.444 it/s
I

In [25]:
# From quant, calculate atomic percentages of elements of interest
spectrum_2.print_concentration_report(selected_elts = ['Sc', 'O', 'Dy'])

+----------+-----------+------------+
| Elements | p0 (at.%) | p0 std (%) |
+----------+-----------+------------+
| O        |    57.527 |      0.313 |
| Sc       |    18.948 |      0.211 |
| Dy       |    23.525 |      0.142 |
+----------+-----------+------------+

Disclaimer : The presented errors correspond to the statistical error on the fitted intensity of the peaks.
In other words it corresponds to the precision of the measurment.
The accuracy of the measurment strongly depends on other factors such as absorption, cross-sections, etc...
Please consider these parameters when interpreting the results.


In [20]:
# Look at quality of fit of spectral model to original spectrum
plot_data_model_ROI(spectrum_2)

In [24]:
est.W_[:-2,:].sum()

0.9999977142936107

In [57]:
get_xray_lines_near_energy(1.3)

['Mg_Kb',
 'Ge_Lb3',
 'Br_Ll',
 'Sm_Mg',
 'Dy_Ma',
 'Nd_M2N4',
 'As_Lb1',
 'As_La',
 'Hf_Mz',
 'Dy_Mb',
 'Ta_Mz',
 'Tb_Mb',
 'Br_Ln',
 'Mg_Ka',
 'Eu_Mg',
 'Ho_Ma',
 'Pm_M2N4',
 'Se_Ln',
 'Pm_Mg',
 'Tb_Ma',
 'Lu_Mz',
 'Pr_M2N4',
 'Se_La',
 'Ge_Lb1',
 'W_Mz',
 'As_Lb3',
 'Kr_Ll',
 'Ho_Mb',
 'Gd_Mb',
 'Se_Ll']

#### Tailor quant by splitting lines

In [61]:
# Split selected X-ray lines at chosen X-ray energy
#spectrum_2.build_G(elements_dict = {'21' : 3.0, '29' : 3.0, '66' : 3.0})
spectrum_2.build_G()

Input elements
[6, 8, 21, 29, 66]
Valid elements
[6, 8, 21, 29, 66]
[2.90280224e-22]
6
[4.88615587e-22]
8
[8.26024096e-22]
21
[1.44764156e-21]
29
[8.1000302e-22]
66


In [62]:
spectrum_2.set_fixed_W(phases_dict = {'p0' : {'O' : 0.6}})

array([[-1. ],
       [ 0.6],
       [-1. ],
       [-1. ],
       [-1. ],
       [-1. ],
       [-1. ]])

In [63]:
myW = spectrum_2.set_fixed_W(phases_dict = {'p0' : {'O' : 0.6}})

In [64]:
# Re-run quant with split lines
#est = espm.estimators.SmoothNMF(n_components = 1, tol = 1e-6, max_iter = 500, G = spectrum_2.model, hspy_comp = True)
est = espm.estimators.SmoothNMF(n_components = 1, tol = 1e-6, max_iter = 500, G = spectrum_2.model, hspy_comp = True,
                                fixed_W = myW, simplex_W=True)
spectrum_2.decomposition(algorithm = est)

It 10 / 500: loss 1.236745e+02,  527.931 it/s
It 20 / 500: loss 1.164764e+02,  489.920 it/s
It 30 / 500: loss 9.708410e+01,  544.771 it/s
It 40 / 500: loss 3.302253e+01,  540.526 it/s
It 50 / 500: loss 2.271369e+01,  570.723 it/s
It 60 / 500: loss 1.274082e+01,  593.700 it/s
It 70 / 500: loss 1.242968e+01,  595.937 it/s
It 80 / 500: loss 1.240101e+01,  616.855 it/s
It 90 / 500: loss 1.239491e+01,  631.211 it/s
exits because of relative change < tol: 9.95173723465243e-07
Stopped after 93 iterations in 0.0 minutes and 0.0 seconds.
Decomposition info:
  normalize_poissonian_noise=False
  algorithm=SmoothNMF()
  output_dimension=None
  centre=None
scikit-learn estimator:
SmoothNMF()


In [45]:
# Look at quality of fit of spectral model to original spectrum
plot_data_model_ROI(spectrum_2)

In [65]:
# From quant, calculate atomic percentages of elements of interest
spectrum_2.print_concentration_report(selected_elts = ['C', 'O', 'Sc', 'Cu', 'Dy'])

+----------+-----------+-------------+
| Elements | p0 (at.%) |  p0 std (%) |
+----------+-----------+-------------+
| C        |     0.000 | 2760610.477 |
| O        |    48.274 |       0.270 |
| Sc       |    22.511 |       0.215 |
| Cu       |     5.687 |       0.320 |
| Dy       |    23.528 |       0.144 |
+----------+-----------+-------------+

Disclaimer : The presented errors correspond to the statistical error on the fitted intensity of the peaks.
In other words it corresponds to the precision of the measurment.
The accuracy of the measurment strongly depends on other factors such as absorption, cross-sections, etc...
Please consider these parameters when interpreting the results.


In [66]:
est.W_

array([[1.00000000e-14],
       [6.00000000e-01],
       [2.79788516e-01],
       [7.06840596e-02],
       [2.92427771e-01],
       [4.76089807e-02],
       [6.78683348e-02]])

#### Tailor quant by refining spectrum calibration

In [150]:
# Find X-ray table, for instance C/Users/LSME/.conda/envs/espm_112/Lib/site-packages/espm/tables
# Open up chosen X-ray table using e.g. a text editor
# Use calibrate function to calibrate off chosen peaks
# Might need to plot spectrum before this function runs
# spectrum_2.plot()
spectrum_2.calibrate()

C:\Users\LSME\.conda\envs\espm_112\Lib\site-packages\jupyter_client\session.py:721: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant: nan
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


In [151]:
spectrum_2.axes_manager

Navigation axis name,size,index,offset,scale,units
x,2,0,0.0,0.011020250601305244,nm
y,2,0,0.0,0.011020250601305244,nm
Signal axis name,size,,offset,scale,units
X-ray energy,2048,,0.19488341708542717,0.005000251256281407,keV


#### Try tailoring the slope and intercept for peak widths

In [145]:
# start from CH values of slope 0.0117 and intercept 0.065
# closest have so far is slope 0.015 and intercept 0.05
spectrum_2.set_analysis_parameters(beam_energy=200,elements = ['C','O', 'Sc', 'Cu', 'Dy'],
                                  detector_type='SDD_efficiency.txt',
                                  xray_db='200keV_xrays.json',
                                   width_slope = 0.015,
                                  width_intercept = 0.05)

#### Try creating tailored X-ray table

In [170]:
# Define customised table
# Modify Sc peak positions by adding 0.005 keV to major K lines
spectrum_2.set_analysis_parameters(beam_energy=200,elements = ['C','O', 'Sc', 'Cu', 'Dy'],
                                  detector_type='SDD_efficiency.txt',
                                  xray_db='200keV_xrays_REoxides.json',
                                   width_slope = 0.015,
                                  width_intercept = 0.05)

#### Add in prior knowledge of sample density and thickness

In [164]:
# from EELS estimated t = 65 nm = 65e-7 cm
# from database density = 6.9 g cm-3
spectrum_2.set_analysis_parameters(beam_energy=200,elements = ['C','O', 'Sc', 'Cu', 'Dy'],
                                  detector_type='SDD_efficiency.txt',
                                   thickness = 65e-7,
                                   density = 6.9,
                                  xray_db='200keV_xrays_REoxides.json',
                                   width_slope = 0.015,
                                  width_intercept = 0.05)

#### Command for closing all plot windows

In [31]:
plt.close("all")

#### Doing some testing